In [8]:
# Basic data cleaning and exploration of the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import textblob as tb
import nltk as nltk

In [9]:
os.chdir('C:\\Users\\Mr. USER\\AppData\\Local\\Programs\\Python\Python311')

In [15]:
df = pd.read_csv('cleaned_Airline_review_Final.csv')
df.head()

,Review ID,Airline Name,Overall_Rating,Review_Title,Review_Date,Review Date,Verified,Review,Aircraft,Type Of Traveller,...,Date Flown,Seat Comfort,Cabin Staff Service,Food_Beverages,Ground Service,Inflight Entertainment,Wifi_Connectivity,Value For Money,Recommended,IATA CODE
0,3801,Alaska Airlines,1,"""carpet was so dirty""",2023-07-24,24th July 2023,False,"Never again! My ticket was messed up, I had t...",Boeing 737-800,Solo Leisure,...,7/1/2023,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no,AS
1,3802,Alaska Airlines,1,"""have the audacity to be truthful""",2023-07-23,23rd July 2023,True,Booked a round trip with this company for a ...,NaN,Family Leisure,...,7/1/2023,1.0,1.0,1.0,1.0,1.0,4.0,1.0,no,AS
2,3803,Alaska Airlines,2,"""a lot of shortfalls all over the place""",2023-07-22,22nd July 2023,True,No in flight entertainment. Inconvenient foo...,Boeing 737,Solo Leisure,...,7/1/2023,1.0,2.0,2.0,1.0,1.0,1.0,2.0,no,AS
3,3804,Alaska Airlines,1,"""Absolutely terrible experience""",2023-07-22,22nd July 2023,True,Absolutely terrible experience flying with A...,NaN,Family Leisure,...,7/1/2023,1.0,1.0,2.0,1.0,2.0,2.0,1.0,no,AS
4,3805,Alaska Airlines,1,"""the pilot had no sense of urgency""",2023-07-21,21st July 2023,True,We were on a connecting flight to London and ...,NaN,Business,...,7/1/2023,3.0,5.0,5.0,1.0,NaN,NaN,1.0,no,AS


In [16]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.probability import FreqDist

# Ensure you have downloaded the necessary NLTK resources
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def get_wordnet_pos(treebank_tag):
    """Converts treebank tags to wordnet tags."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default, if mapping is not possible

def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w.lower() for w in tokens if w.lower() not in stop_words and w.isalpha()]  # Also remove non-alphabetic tokens
    # Part-of-Speech Tagging for Lemmatization
    pos_tags = pos_tag(filtered_tokens)
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag)) for word, tag in pos_tags]
    return lemmatized_tokens





[nltk_data] Downloading package punkt to C:\Users\Mr.
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mr.
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mr. USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Mr.
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
# Example DataFrame load and preprocessing
# df = pd.read_csv('path_to_your_file.csv')
df['tokenized_review'] = df['Review'].apply(preprocess_text)
df['tokenized_review'].head(10)


0    [never, ticket, mess, take, later, flight, sit...
1    [book, round, trip, company, family, er, first...
2    [flight, entertainment, inconvenient, food, or...
3    [absolutely, terrible, experience, fly, alaska...
4    [connect, flight, london, pilot, sense, urgenc...
5    [week, visit, family, washington, two, child, ...
6    [partner, travel, seattle, back, home, san, fr...
7    [first, time, fly, alaska, airline, say, enoug...
8    [rude, people, phone, refuse, supply, info, ne...
9    [alaska, airline, opinion, one, worst, airline...
Name: tokenized_review, dtype: object

In [14]:
# To calculate frequency distribution of words across all reviews
all_words = [word for tokens_list in df['tokenized_review'] for word in tokens_list]
fdist = FreqDist(all_words)
print(fdist.most_common(20))

[('flight', 2798), ('airline', 1195), ('get', 1077), ('hour', 848), ('time', 804), ('fly', 762), ('u', 685), ('service', 605), ('delay', 588), ('customer', 571), ('seat', 558), ('go', 552), ('would', 546), ('one', 473), ('say', 461), ('tell', 456), ('plane', 455), ('day', 453), ('take', 450), ('cancel', 443)]


In [27]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Download VADER lexicon if not already done
nltk.download('vader_lexicon')

# Initialize the SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

def detailed_sentiment_analysis(text):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    
    # Initialize lists to store sentiment scores of individual sentences
    sentence_tb_scores = []
    sentence_vader_scores = []
    
    # Analyze sentiment of each sentence
    for sentence in sentences:
        # TextBlob Analysis for the sentence
        blob = TextBlob(sentence)
        tb_score = blob.sentiment.polarity
        sentence_tb_scores.append(tb_score)
        
        # Vader Analysis for the sentence
        vader_score = sid.polarity_scores(sentence)['compound']
        sentence_vader_scores.append(vader_score)
    
    # Calculate average sentiment scores for all sentences
    avg_tb_score = sum(sentence_tb_scores) / len(sentence_tb_scores)
    avg_vader_score = sum(sentence_vader_scores) / len(sentence_vader_scores)
    
    # Averaging the scores
    average_score = (avg_tb_score + avg_vader_score) / 2
    
    # Determine the sentiment label based on the averaged score
    sentiment_label = 'Positive' if average_score > 0.05 else 'Negative' if average_score < -0.05 else 'Neutral'
    
    return average_score, sentiment_label

# Apply the function to the DataFrame
df['sentiment_score'], df['sentiment_label'] = zip(*df['Review'].apply(detailed_sentiment_analysis))

# Display the DataFrame with sentiment analysis results
print(df[['Review', 'sentiment_score', 'sentiment_label']].head())


[nltk_data] Downloading package vader_lexicon to C:\Users\Mr.
[nltk_data]     USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [31]:
#add sentiment score and sentiment label column to the dataframe containing Review ID	Airline Name	Overall_Rating	Review_Title	Review_Date	Review Date	Verified	Review	Aircraft	Type Of Traveller	...	Date Flown	Seat Comfort	Cabin Staff Service	Food_Beverages	Ground Service	Inflight Entertainment	Wifi_Connectivity	Value For Money	Recommended	IATA CODE
df['sentiment_score'], df['sentiment_label'] = zip(*df['Review'].apply(detailed_sentiment_analysis))

# Display the DataFrame with sentiment analysis results
print(df[['Review ID', 'Airline Name', 'Overall_Rating', 'Review_Title', 'Review_Date', 'Review Date', 'Verified', 'Review', 'Aircraft', 'Type Of Traveller', 'sentiment_score', 'sentiment_label']].head(10))




   Review ID     Airline Name  Overall_Rating                              Review_Title Review_Date     Review Date  Verified  \
0       3801  Alaska Airlines               1                     "carpet was so dirty"  2023-07-24  24th July 2023     False   
1       3802  Alaska Airlines               1        "have the audacity to be truthful"  2023-07-23  23rd July 2023      True   
2       3803  Alaska Airlines               2  "a lot of shortfalls all over the place"  2023-07-22  22nd July 2023      True   
3       3804  Alaska Airlines               1          "Absolutely terrible experience"  2023-07-22  22nd July 2023      True   
4       3805  Alaska Airlines               1       "the pilot had no sense of urgency"  2023-07-21  21st July 2023      True   
5       3806  Alaska Airlines               1   "employees do not care about customers"  2023-07-18  18th July 2023     False   
6       3807  Alaska Airlines               1      "Worst flight experience of my life"  2023-07-

In [32]:
# export the dataframe to a csv file
df.to_csv('sentiment_analysis_Airline_review.csv', index=False)